## Long Range Transportation Plan

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp_data_analysis.sql import to_snakecase
from shared_utils import utils
import _harmonization_utils as harmonization_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_833/1040096962.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  im

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### SANDAG
* https://www.arcgis.com/home/item.html?id=ba8b2a6247ef463cb7411f039959f3ee
* https://www.arcgis.com/apps/mapviewer/index.html?webmap=ba8b2a6247ef463cb7411f039959f3ee

In [3]:
columns_to_keep = [
        "project_title",
        "lead_agency",
        "project_year",
        "project_category",
        "grant_program",
        "project_description",
        "total_project_cost",
        "location",
         "city",
        "county",
        "data_source",
        "notes",
    ]


In [43]:
def create_notes(df, note_cols:list):
    prefix = "_" 
    for column in note_cols:
        df[f"{prefix}{column}"] = df[column].astype(str)
    note_cols = [prefix + sub for sub in note_cols]
    
    # https://stackoverflow.com/questions/65532480/how-to-combine-column-names-and-values
    def combine_notes(x):
        return ', '.join([col + ': ' + x[col] for col in note_cols])
    df['notes'] = df.apply(combine_notes, axis = 1)
    df.notes = df.notes.str.replace("_", " ")
    
    return df 

In [5]:
def harmonizing_lrtp(
    df,
    agency_name_col: str,
    project_name_col: str,
    project_description_col: str,
    project_category_col: str,
    project_cost_col: str,
    location_col: str,
    county_col: str,
    city_col: str,
    project_year_col: str,
    program_col: str,
    data_source: str,
    note_cols: list,
    cost_in_millions: bool = True,
):
    """
    Take a dataset and change the column names/types to
    default names and formats.
    
    Add metric if the project is fully funded or not.
    """
    rename_columns = {
        agency_name_col: "lead_agency",
        project_name_col: "project_title",
        project_description_col: "project_description",
        project_category_col: "project_category",
        project_cost_col: "total_project_cost",
        location_col: "location",
        county_col: "county",
        city_col: "city",
        project_year_col: "project_year",
        program_col: "grant_program"
    }
    # Rename columns
    df = df.rename(columns=rename_columns)

    # Add data source
    df["data_source"] = data_source

    # Divide cost columns by millions
    # If bool is set to True
    if cost_in_millions:
        for i in cost_columns:
            df[i] = df[i].divide(1_000_000)

    # Create columns even if they don't exist, just to harmonize
    # before concatting.
    create_columns = ["county","city","notes", "project_year", "project_category"]
    for column in create_columns:
        if column not in df:
            df[column] = "None"
    if "grant_program" not in df:
        df["grant_program"] = data_source
    
    # Create notes
    df = create_notes(df, note_cols)

    # Fill in any nulls
    df = df.fillna(df.dtypes.replace({"float64": 0.0, "object": "None"}))
    
    df = df[columns_to_keep]
    
    return df

In [6]:
def open_rest_server(url_pt_1: str, url_pt_2: str, layer_name: list):
    full_gdf = pd.DataFrame()
    for i in layer_name:
        gdf = to_snakecase(gpd.read_file(f"{url_pt_1}{i}{url_pt_2}"))
        gdf["layer_name"] = i
        full_gdf = pd.concat([full_gdf, gdf], axis=0)

    return full_gdf

In [7]:
sandag_columns_keep = [
    "type",
    "caption",
    "phase",
    "year_built",
    "category",
    "project_name",
    "description",
    "cost2020m",
    "aqc_2016_1",
    "aqc_2020_1",
    "pricmcp",
    "conncmcp",
    "status",
    "geometry",
    "layer_name",
    "corridor_i",
    "type_1",
    "existing",
    "limits",
    "description_1",
    "route",
    "routetype",
    "route_desc",
    "rp_2021_id",
    "rp_2021_id_1",
    "capital_cost___2020__millions",
]

In [8]:
def sandag_lrtp():
    sandag_layers_list = [
    "Complete_Corridors_Connectors_Intersections_Ramps_RP2021",
    "Complete_Corridors_Highway_RP2021",
    "Transit_Leap_RP2021",
    "Adopted_Regional_Bike_Network_RP2021",
    "Mobility_Hubs_and_Flexible_Fleets",
    "Complete_Corridors_Regional_Arterials",
    "Goods_Movement"]
    
    sandag_url_pt1 = "https://services1.arcgis.com/HG80xaIVT1z1OdO5/ArcGIS/rest/services/"
    sandag_url_pt2 = "/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*+&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="
    sandag = open_rest_server(sandag_url_pt1,sandag_url_pt2,sandag_layers_list)
    sandag = sandag[sandag_columns_keep]
    
    # Create columns
    sandag["lead_agency"] = "SANDAG"
    sandag["county"] = "San Diego"

    return sandag

In [9]:
sandag_rp_21 = sandag_lrtp()

In [10]:
sandag_rp_21.layer_name.nunique()

7

In [11]:
sandag_rp_21.layer_name.value_counts()

Adopted_Regional_Bike_Network_RP2021                        185
Transit_Leap_RP2021                                          70
Complete_Corridors_Connectors_Intersections_Ramps_RP2021     64
Complete_Corridors_Highway_RP2021                            56
Mobility_Hubs_and_Flexible_Fleets                            31
Goods_Movement                                                9
Complete_Corridors_Regional_Arterials                         1
Name: layer_name, dtype: int64

In [12]:
sandag_rp_21.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 416 entries, 0 to 8
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   type                           64 non-null     object  
 1   caption                        57 non-null     object  
 2   phase                          384 non-null    object  
 3   year_built                     384 non-null    object  
 4   category                       351 non-null    object  
 5   project_name                   375 non-null    object  
 6   description                    382 non-null    object  
 7   cost2020m                      186 non-null    object  
 8   aqc_2016_1                     171 non-null    object  
 9   aqc_2020_1                     171 non-null    object  
 10  pricmcp                        167 non-null    object  
 11  conncmcp                       165 non-null    object  
 12  status                        

In [13]:
sandag_rp_21.drop(columns = ['geometry']).sample(1)

,type,caption,phase,year_built,category,project_name,description,cost2020m,aqc_2016_1,aqc_2020_1,pricmcp,conncmcp,status,layer_name,corridor_i,type_1,existing,limits,description_1,route,routetype,route_desc,rp_2021_id,rp_2021_id_1,capital_cost___2020__millions,lead_agency,county
33,NaN,NaN,2035,2035,Active Transportation,Coastal Rail Trail San Diego - Mission Bay (Clairemont to Tecolote),Off-Street and On-Street,NaN,NaN,NaN,NaN,NaN,Proposed,Adopted_Regional_Bike_Network_RP2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AT034,AT034,$15,SANDAG,San Diego


In [44]:
def harmonize_lrtp():
    df = sandag_lrtp()
    df = harmonizing_lrtp(
    df,
    agency_name_col= "lead_agency",
    project_name_col= "project_name",
    project_description_col= "description",
    project_category_col= "category",
    project_cost_col= "cost2020m",
    location_col= "geometry",
    county_col= "county",
    city_col= "",
    project_year_col= "",
    program_col= "",
    data_source= "SANDAG LRTP",
    note_cols= ['year_built', 'aqc_2016_1', 'aqc_2020_1', 'pricmcp',
       'conncmcp', 'status', 'layer_name', 'corridor_i', 'type_1',
       'existing', 'limits', 'description_1', 'route', 'routetype',
       'route_desc', 'rp_2021_id', 'rp_2021_id_1',
       'capital_cost___2020__millions'], 
    cost_in_millions=False)
        
    return df

In [45]:
lrtp_cleaned = harmonize_lrtp()

In [40]:
type(lrtp_cleaned)

geopandas.geodataframe.GeoDataFrame

In [46]:
lrtp_cleaned.head()

,project_title,lead_agency,project_year,project_category,grant_program,project_description,total_project_cost,location,city,county,data_source,notes
0,I-5 (SR 78),SANDAG,None,Complete Corridor: MLC,SANDAG LRTP,"South to East and West to North, North to East and West to South",$352,POINT (-117.35126 33.17707),None,San Diego,SANDAG LRTP,"year built: 2035, aqc 2016 1: 2040, aqc 2020 1: 2040, pricmcp: North County Corridor, conncmcp: N/A, status: Proposed, layer name: Complete Corridors Connectors Intersections Ramps RP2021, corridor i: nan, type 1: nan, existing: nan, limits: nan, description 1: nan, route: nan, routetype: nan, route desc: nan, rp 2021 id: nan, rp 2021 id 1: nan, capital cost 2020 millions: nan"
1,I-15 (SR 78),SANDAG,None,Complete Corridor: MLC,SANDAG LRTP,South to West and East to North,$147,POINT (-117.10484 33.12706),None,San Diego,SANDAG LRTP,"year built: 2050, aqc 2016 1: 2050, aqc 2020 1: 2050, pricmcp: I-15, conncmcp: North County Corridor, status: Proposed, layer name: Complete Corridors Connectors Intersections Ramps RP2021, corridor i: nan, type 1: nan, existing: nan, limits: nan, description 1: nan, route: nan, routetype: nan, route desc: nan, rp 2021 id: nan, rp 2021 id 1: nan, capital cost 2020 millions: nan"
2,I-5 (I-805),SANDAG,None,Complete Corridor: MLC,SANDAG LRTP,North to North and South to South,$84,POINT (-117.22515 32.90627),None,San Diego,SANDAG LRTP,"year built: 2035, aqc 2016 1: 2040, aqc 2020 1: 2040, pricmcp: SB2S, conncmcp: N/A, status: Proposed, layer name: Complete Corridors Connectors Intersections Ramps RP2021, corridor i: nan, type 1: nan, existing: nan, limits: nan, description 1: nan, route: nan, routetype: nan, route desc: nan, rp 2021 id: nan, rp 2021 id 1: nan, capital cost 2020 millions: nan"
3,I-805 (SR 52),SANDAG,None,Complete Corridor: MLC,SANDAG LRTP,North to West and East to South,$126,POINT (-117.17820 32.84756),None,San Diego,SANDAG LRTP,"year built: 2050, aqc 2016 1: 2050, aqc 2020 1: 2050, pricmcp: SB2S, conncmcp: CCT, status: Proposed, layer name: Complete Corridors Connectors Intersections Ramps RP2021, corridor i: nan, type 1: nan, existing: nan, limits: nan, description 1: nan, route: nan, routetype: nan, route desc: nan, rp 2021 id: nan, rp 2021 id 1: nan, capital cost 2020 millions: nan"
4,I-15 (SR 52),SANDAG,None,Complete Corridor: MLC,SANDAG LRTP,West to North and South to East,$181,POINT (-117.12061 32.84182),None,San Diego,SANDAG LRTP,"year built: 2035, aqc 2016 1: 2040, aqc 2020 1: 2040, pricmcp: I-15, conncmcp: CCT, SB2S, status: Proposed, layer name: Complete Corridors Connectors Intersections Ramps RP2021, corridor i: nan, type 1: nan, existing: nan, limits: nan, description 1: nan, route: nan, routetype: nan, route desc: nan, rp 2021 id: nan, rp 2021 id 1: nan, capital cost 2020 millions: nan"


### MTC
* 133 projects but point and line map returns 240 ish rows.
* Projects that are in different locations can have the same name split by open year...
* I guess the geospatial projects are split into various phases while the regular pandas df is just the project in its entirety.
* https://www.planbayarea.org/2050-plan/final-plan-bay-area-2050/final-supplemental-reports/interactive-transportation-project-list
* https://opendata.mtc.ca.gov/datasets/MTC::2023-transportation-improvement-program-projects-point/about
* https://opendata.mtc.ca.gov/datasets/MTC::plan-bay-area-2050-transportation-projects-line/about
* https://data.bayareametro.gov/Projects/Plan-Bay-Area-2050-Transportation-Project-List/y7ka-jksz

In [17]:
mtc_layer_list = ["planbayarea2050_rtp_line", "planbayarea2050_rtp_point"]

In [18]:
mtc_url_pt_1 = "https://services3.arcgis.com/i2dkYWmb4wHvYPda/ArcGIS/rest/services/"

In [19]:
mtc_url_pt_2 = "/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*+&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

In [47]:
mtc = open_rest_server(mtc_url_pt_1, mtc_url_pt_2, mtc_layer_list)

In [53]:
mtc[mtc.proj_title == "Rapid Bus | AC Transit | Modernization"].drop(columns = ['geometry'])

,objectid,map_id,subcompone,open_year,strategy,rtp_id,proj_title,mode_,mode_detai,investment,shape__length,layer_name
62,63,MU-029,AC Transit Rapid Network: Frequency Boost,2021 to 2035,"T10. Enhance Local Transit Frequency, Capacity & Reliability",21-T10-073,Rapid Bus | AC Transit | Modernization,Transit,Local Bus,Frequency Boost,1.82,planbayarea2050_rtp_line
63,64,MU-030,AC Transit Rapid Network: Grand Ave,2021 to 2035,"T10. Enhance Local Transit Frequency, Capacity & Reliability",21-T10-073,Rapid Bus | AC Transit | Modernization,Transit,Local Bus,Frequency Boost,0.38,planbayarea2050_rtp_line
64,65,MU-031,AC Transit Rapid Network: Foothill Blvd,2021 to 2035,"T10. Enhance Local Transit Frequency, Capacity & Reliability",21-T10-073,Rapid Bus | AC Transit | Modernization,Transit,Local Bus,Frequency Boost,0.22,planbayarea2050_rtp_line
65,66,MU-032,AC Transit Rapid Network: Line 12,2021 to 2035,"T10. Enhance Local Transit Frequency, Capacity & Reliability",21-T10-073,Rapid Bus | AC Transit | Modernization,Transit,Local Bus,Frequency Boost,0.15,planbayarea2050_rtp_line
66,67,MU-033,AC Transit Rapid Network: Telegraph Ave,2036 to 2050,"T10. Enhance Local Transit Frequency, Capacity & Reliability",21-T10-073,Rapid Bus | AC Transit | Modernization,Transit,Local Bus,Frequency Boost,0.11,planbayarea2050_rtp_line
67,68,MU-034,AC Transit Rapid Network: MacArthur Blvd,2036 to 2050,"T10. Enhance Local Transit Frequency, Capacity & Reliability",21-T10-073,Rapid Bus | AC Transit | Modernization,Transit,Local Bus,Frequency Boost,0.22,planbayarea2050_rtp_line
68,69,MU-035,AC Transit Rapid Network: Hesperian Blvd,2036 to 2050,"T10. Enhance Local Transit Frequency, Capacity & Reliability",21-T10-073,Rapid Bus | AC Transit | Modernization,Transit,Local Bus,Frequency Boost,0.05,planbayarea2050_rtp_line
69,70,MU-036,AC Transit Rapid Network: Broadway,2036 to 2050,"T10. Enhance Local Transit Frequency, Capacity & Reliability",21-T10-073,Rapid Bus | AC Transit | Modernization,Transit,Local Bus,Frequency Boost,0.06,planbayarea2050_rtp_line


In [21]:
mtc2 = mtc.drop(columns = ['shape__length','objectid', 'map_id',"investment", "subcompone", "open_year", "mode_detai"])

In [22]:
mtc_dissolve = mtc2.dissolve(by=["layer_name", "rtp_id","proj_title", "strategy", "mode_",]).reset_index()

In [23]:
mtc_dissolve.sample()

,layer_name,rtp_id,proj_title,strategy,mode_,geometry
61,planbayarea2050_rtp_line,21-T11-101,Rail | Modernization & Electrification | Caltrain/High Speed Rail | San Francisco to San Jose,T11. Expand and Modernize the Regional Rail Network,Transit,"LINESTRING (-121.87166 37.29713, -121.87400 37.29929, -121.87960 37.30627, -121.88468 37.31270, -121.88792 37.31664, -121.88962 37.31775, -121.89256 37.31816, -121.89623 37.31835, -121.89844 37.31920, -121.90119 37.32180, -121.90258 37.32357, -121.90327 37.33185, -121.90346 37.33316, -121.90447 37.33452, -121.91033 37.33957, -121.91294 37.34143, -121.91518 37.34270, -121.91979 37.34498, -121.93633 37.35351, -121.95399 37.36112, -121.96884 37.36757, -121.97412 37.36975, -121.97783 37.37048, -121.99607 37.37050, -122.00038 37.37075, -122.01405 37.37352, -122.02023 37.37514, -122.03079 37.37862, -122.04480 37.38324, -122.05757 37.38745, -122.07611 37.39456, -122.09759 37.40301, -122.10454 37.40579, -122.10720 37.40726, -122.11460 37.41203, -122.13074 37.42224, -122.14192 37.42922, -122.15632 37.43830, -122.16537 37.44377, -122.17599 37.45083, -122.18267 37.45495, -122.19795 37.46462, -122.20996 37.47230, -122.21859 37.47784, -122.23166 37.48588, -122.23942 37.49076, -122.24326 37.49330, -122.25217 37.50096, -122.25985 37.50770, -122.26657 37.51340, -122.26931 37.51575, -122.27200 37.51771, -122.27650 37.52135, -122.28197 37.52593, -122.28418 37.52753, -122.28966 37.53124, -122.29429 37.53447, -122.29708 37.53747, -122.30219 37.54335, -122.30785 37.54994, -122.30910 37.55215, -122.31094 37.55592, -122.31302 37.55815, -122.32060 37.56504, -122.32800 37.57195, -122.33081 37.57399, -122.33472 37.57582, -122.34070 37.57836, -122.34506 37.58016, -122.36257 37.58756, -122.37345 37.59206, -122.38057 37.59514, -122.38373 37.59699, -122.38701 37.60021, -122.39585 37.60881, -122.40173 37.61532, -122.40726 37.62285, -122.41126 37.62978, -122.41308 37.63367, -122.41312 37.63792, -122.41302 37.64400, -122.41192 37.64725, -122.41014 37.65014, -122.40772 37.65303, -122.40362 37.65729, -122.39649 37.66485, -122.39190 37.66950, -122.39003 37.67202, -122.38952 37.67400, -122.38978 37.67575, -122.39061 37.67762, -122.39164 37.67888, -122.39902 37.68771, -122.40100 37.69102, -122.40181 37.69515, -122.40184 37.70150, -122.40186 37.70761, -122.40166 37.71032, -122.40061 37.71363, -122.39795 37.72107, -122.39660 37.72514, -122.39602 37.72949, -122.39497 37.73707, -122.39383 37.74489, -122.39289 37.75216, -122.39239 37.75531, -122.39239 37.75759, -122.39269 37.76359, -122.39349 37.76539, -122.39482 37.76675, -122.39677 37.76836, -122.39960 37.77053, -122.39999 37.77114, -122.40005 37.77175, -122.40001 37.77226, -122.39956 37.77280, -122.39879 37.77333, -122.39692 37.77480, -122.39475 37.77657)"


In [24]:
mtc_dissolve.proj_title.nunique()

99

In [25]:
mtc_all_projects = pd.read_json('https://data.bayareametro.gov/resource/y7ka-jksz.json?$query=SELECT%0A%20%20%60plan_strategy%60%2C%0A%20%20%60rtpid%60%2C%0A%20%20%60title%60%2C%0A%20%20%60scope%60%2C%0A%20%20%60open_period%60%2C%0A%20%20%60funding_millions_yoe%60%2C%0A%20%20%60county%60')

In [26]:
mtc_all_projects[mtc_all_projects.title == "Rapid Bus | AC Transit | Modernization"]

,plan_strategy,rtpid,title,scope,open_period,funding_millions_yoe,county
72,Local Transit,21-T10-073,Rapid Bus | AC Transit | Modernization,"This program includes funding to implement rapid transit improvements to existing bus service. Improvements include new rapid bus service; improved bus stops and stations; new/improved transit signal priority (including on-street and on-bus equipment); transit priority infrastructure; dedicated bus lanes; queue jumps; and frequency upgrades (5-12 minute peak headways on routes 18, 20/21, 40, 57, 97 and NL).",Various,2000,Various


In [27]:
mtc_dissolve.proj_title.nunique(), mtc_all_projects.title.nunique()

(99, 133)

In [28]:
mtc_dissolve.shape

(114, 6)

In [30]:
mtc2 = pd.merge(mtc_dissolve, mtc_all_projects, how = "left", left_on = ["proj_title"], right_on = ["title"])

In [31]:
mtc2.shape, mtc2.proj_title.nunique()

((114, 13), 99)

In [32]:
mtc2.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 114 entries, 0 to 113
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   layer_name            114 non-null    object  
 1   rtp_id                114 non-null    object  
 2   proj_title            114 non-null    object  
 3   strategy              114 non-null    object  
 4   mode_                 114 non-null    object  
 5   geometry              114 non-null    geometry
 6   plan_strategy         114 non-null    object  
 7   rtpid                 114 non-null    object  
 8   title                 114 non-null    object  
 9   scope                 114 non-null    object  
 10  open_period           114 non-null    object  
 11  funding_millions_yoe  114 non-null    int64   
 12  county                114 non-null    object  
dtypes: geometry(1), int64(1), object(11)
memory usage: 12.5+ KB


In [33]:
# mtc2.explore('proj_title', legend= False)

In [34]:
# mtc.explore("layer_name")

In [35]:
# mtc[mtc.proj_title == "Rapid Bus | AC Transit | Modernization"].explore("objectid")

### Tahoe
* Open data portal: https://www.tahoeopendata.org/search?tags=transportation
* https://www.tahoeopendata.org/datasets/TRPA::2020-regional-transportation-plan-projects/about

In [37]:
def tmpo_lrtp():
    tahoe_url = "https://maps.trpa.org/server/rest/services/Datadownloader_Transportation/MapServer/19/query?where=1%3D1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*+&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson"
    gdf = to_snakecase(gpd.read_file(tahoe_url))
    gdf.complete_year = gdf.complete_year.astype(int)
    
    # Filter out projects that are not yet completed
    gdf2 = gdf[gdf.complete_year > 2022].reset_index(drop = True)
    
    gdf2["county
    return gdf2

In [38]:
tahoe = tmpo_lrtp()

In [54]:
tahoe.head().drop(columns = ['geometry'])

,objectid,eip,project_name,implementer,start_year,complete_year,funding_type,estimated_cost,total_cost,category,plan_name,label,description,financial_status,url,phase,shape_starea__,shape_stlength__
0,1,03.01.02.0017,"Tallac Historic Site, Valhalla, and the Visitor Center Improvements",U.S. Forest Service - Lake Tahoe Basin Management Unit,2013,2027,Capital,"$11,100,000",,Corridor Revitalization/Complete Streets,,Tallac Historic Site Improvements,"The objective of the overall project is to complete design and implementation of water quality protection BMPs and upgrades to meet recreational use standards at the Tallac Historic Site, Valhalla and Visitors Center. Upgrades can include building rehabilitation, water/sewer improvements and building replacement. Transportation improvements include the creation of a loop road interconnecting various sites, multimodal connections, bike paths and signage.",Constrained,https://eip.laketahoeinfo.org/Project/FactSheet/03.01.02.0017,Implementation,1153845.06,5304.01
1,2,NOT LISTED,Transportation Demand Management Programs,Tahoe Regional Planning Agency,2020,2045,Program,"$180,000",,Technology & Transportation System Management,,Travel Managment Program,Encourage transportation behavior change through incentives and programs.,Constrained,,Implementation,1329738968.17,216949.29
2,3,03.01.02.0102,Improved Parking Management and Wayfinding in Tahoe City,Placer County,2016,2023,Capital,"$2,000,000",,Technology & Transportation System Management,,Tahoe City Parking Management,"This project builds off Tahoe City Downtown Access improvements, located on the constrained project list. Strategies could include consideration of dedicated parking circulators during peak periods, new parking and mobility infrastructure, and wayfinding signage. Wayfinding signage for parking facilities should be incorporated into a comprehensive program for multiple modes.",Constrained,https://eip.laketahoeinfo.org/Project/FactSheet/03.01.02.0102,Implementation,791362.99,10165.33
3,4,01.01.01.0033,Tahoe Valley Greenbelt,City of South Lake Tahoe,2009,2024,Capital,"$8,550,000",,Active Transportation,ATP,Tahoe Valley Greenbelt,"The Greenbelt project includes multi-benefit stormwater, SEZ, bicycle and pedestrian improvements and recreational amenities. Water quality and SEZ enhancements will include improving existing drainage ways and drainage systems to spread, treat, infiltrate, and retain flows from roadways, commercial areas, and other high priority, directly connected urban areas. Pedestrian and bicycle will include improving connectivity within the project area and to regional networks.",Constrained,https://eip.laketahoeinfo.org/Project/FactSheet/01.01.01.0033,Planning/Design,1544760.06,6020.37
4,5,Grouped in Tracker,Various ITS Solutions for the Lake Tahoe Region,Various,2020,2045,Capital,,,Technology & Transportation System Management,,Various ITS Solutions,Internet technology services improvements for the Lake Tahoe transportation region,Unconstrained,,Planning/Design,4142205.77,16438.35
